#### 여기서는 Tutorial에서 배운 개념을 이용하여 간단하게 Network parameter를 분산 환경에서 서로 보내고 가져오는 작업을 해보겠습니다. <br>
    1. 각 actor는 network을 가지고 있다. 
    2. 학습을 하면서 일정 간격으로 actor는 learner의 파라미터를 복제해온다.
    3. actor와 learner의 중간 매개체 역할을 하는 parameter server가 있다.

In [1]:
# !pip install ray

In [2]:
import ray 
import time 
import numpy as np 
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
ray.init() 

2021-01-28 20:20:48,023	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.0.61',
 'raylet_ip_address': '192.168.0.61',
 'redis_address': '192.168.0.61:32428',
 'object_store_address': '/tmp/ray/session_2021-01-28_20-20-47_359586_15089/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-28_20-20-47_359586_15089/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-01-28_20-20-47_359586_15089',
 'metrics_export_port': 58445,
 'node_id': '49928a94a69b5626ab2c92c16da3372aa9d9f12b'}

In [4]:
# QNetwork 정의
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden=32):
        super(QNetwork, self).__init__()

        state_size = state_size[0]
        self.fc1 = nn.Linear(state_size, hidden)
        self.fc2 = nn.Linear(hidden, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Test
state_size = (4, ) 
action_size = 2 
temp_net = QNetwork(state_size, action_size, 32) 
test = torch.randn(size=(4,)) 
temp_net(test), temp_net(test).shape 

(tensor([-0.2422, -0.4731], grad_fn=<AddBackward0>), torch.Size([2]))

In [5]:
@ray.remote
class Parameter_server:
    def __init__(self):
        # model의 parameter 대신에 간단한 list를 활용
        self.params = []
    
    def updates_params(self, model):
        self.params.append(model) 

    def pull_params(self):
        return self.params[-1]

In [6]:
@ray.remote
class Actor:
    def __init__(self, params_server, actor_idx):
        self.params_server = params_server
        self.actor_idx = actor_idx
        
        self.device = 'cpu' # ray는 cpu에서 process가 할당되므로, cuda로 선언하게 되면 작동이 되지 않는다.
        self.actor_network = QNetwork((4, ), 12, 64).to(self.device)
        
    def explore(self):
        while 1:
            time.sleep(5)
            print("Explore..")
            self.pull_params() # parameter server에서 저장된 parameter 가지고 옵니다.
        pass

    def pull_params(self):
        updated_params = ray.get(self.params_server.pull_params.remote()) 
        print("Parameter Type: ", type(updated_params)) 
        self.actor_network.load_state_dict(updated_params) 
        print(f"Model is updated in actor number_{self.actor_idx}")


In [7]:
class Learner: 
    def __init__(self, params_server):
        self.params_server = params_server
        self.device = 'cuda:1' # Learner는 GPU를 써야하므로 CUDA에 할당하기 위해 device를 cuda로 설정
        
        self.q_behave = QNetwork((4, ), 12, 64).to(self.device)
        self.q_target = QNetwork((4, ), 12, 64).to(self.device)
        self.q_target.load_state_dict(self.q_behave.state_dict())
        self.q_target.eval() 

    def push_parameters(self):
        # self.q_behave에서 deepcopy와 .cpu()를 하고 server에 넘겨야, learner가 backward를 통해 업데이트 시, 에러도 나지 않고 잘 넘길 수 있습니다.
        # 그러한 이유는 .cpu()가 in-place 연산이라서 원래 모델 자체를 cpu로 전환해버리기 때문이며, 결국 다시 .cuda() 혹은 .to(self.device) 등으로 cuda에 모델을 도로 올려놓아야 합니다. 
        # 문제는, 이렇게 cpu <-> cuda로 전환하는 시간 비용이 deepcopy보다 더 크기 때문에 속도가 떨어집니다.
        copied_model = deepcopy(self.q_behave).cpu()
        self.params_server.updates_params.remote(copied_model.state_dict())
        
        print("Learner's parameters were sent to parameter server.")
        
    def pull_parameters_from_server(self):
        # parameter server 에 보내기/가져오기가 둘 다 되는지 보기 위해서 파라미터를 가져오는 메소드도 넣어보았습니다.
        # 일반적으로 learner가 가지고 올 일은 없습니다.
        print("Pulling parameters from parameter server.")
        return self.params_server.pull_params.remote() 
    
    def train(self):
        while 1:
            time.sleep(2) # 출력 속도 조절
            self.push_parameters() 
            print("Learner: Model parameter is sent to Server.")

#### 먼저 각 메서드가 잘 작동하는지 테스트를 해보겠습니다

In [8]:
# parameter sever
params_server = Parameter_server.remote()    

# learner와 actor 모두 parameter sever를 상속받아 정의합니다. 
# 그 이유는 learner와 actor는 서로 직접적인 연관이 없도록 유지한 채, parameter server를 통해서 간접적으로 parameter를 주고 받기 위함입니다.
# 이렇게 하는 이유는 ray는 Q-network처럼 cuda에 올려진 객체를 포함한 클래스는 상속받지 못하기 때문입니다. 즉, learner는 ray와 무관하며, 오로지 actor만 ray를 통해 동작합니다. 
learner = Learner(params_server) 
actor = Actor.remote(params_server, 0)   

In [9]:
# model 업데이트하는 것을 임의로 횟수로 잡고, learner의 update 메소드를 그 횟수만큼 실행해봅니다.
iteration = 3
for _  in range(iteration): learner.push_parameters() 

Learner's parameters were sent to parameter server.
Learner's parameters were sent to parameter server.
Learner's parameters were sent to parameter server.


In [ ]:
# learner에서 server의 parameter를 가지고 오는 것도 되는지 test를 해봅니다. (실제로 learner는 server에서 가지고 올 일은 없습니다.)
ray.get(learner.pull_parameters_from_server())  

In [11]:
# actor의 explore 메소드를 통해 server에 위에서 learner로 부터 받은 parameter를 가지고 온 후, actor의 모델로 저장이 되는지 테스트 해봅니다.  
actor.explore.remote()  

ObjectRef(3bf0c856ace5a4d8cb230a570100000001000000)

In [12]:
# 마지막으로 learner가 주기적으로 model parameter를 server에 저장을 하면서,
# 동시에 여러 actor가 explore 메소드를 통해 그 parameter를 지속적으로 가지고 오고 update가 되는지 테스트를 해봅니다.

params_server = Parameter_server.remote()    
learner = Learner(params_server) 

num_actor = 5
for idx in range(num_actor): 
    actor = Actor.remote(params_server, idx)
    actor.explore.remote() 


In [ ]:
learner.train() 

#### 성공을 하게 되면 아래와 같은 메세지를 확인할 수 있습니다. <br>
    1. Learner: Model parameter is sent to Server.  --> 이 메세지는 learner가 server에 parameter를 보낸 것을 성공했을 시의 메세지입니다.
    2. 이 외에, (pid=숫자) 가 붙은 메세지는 각 actor 가 update를 성공했을 때의 메세지입니다.

    [실행 예시]
    
    Learner: Model parameter is sent to Server.
    Learner: Model parameter is sent to Server.
    (pid=87629) Explore..
    (pid=87629) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87629) Model is updated in actor number_3
    (pid=87584) Explore..
    (pid=87584) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87584) Model is updated in actor number_1
    (pid=87630) Explore..
    (pid=87630) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87630) Model is updated in actor number_0
    (pid=87588) Explore..
    (pid=87588) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87588) Model is updated in actor number_4
    (pid=87582) Explore..
    (pid=87582) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87582) Model is updated in actor number_2
    Learner: Model parameter is sent to Server.
    Learner: Model parameter is sent to Server.
    Learner: Model parameter is sent to Server.
    (pid=87629) Explore..
    (pid=87629) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87629) Model is updated in actor number_3
    (pid=87584) Explore..
    (pid=87584) Parameter Type:  <class 'collections.OrderedDict'>
    (pid=87584) Model is updated in actor number_1